In [1]:
__file__=''
import os,sys
import pandas as pd
sys.path.append(os.path.join(os.path.dirname(__file__),'../LIB/'))
sys.path.append(os.path.join(os.path.dirname(__file__),'../../../../automl/automl_libs/'))
from env import FILE
import utils
from itertools import combinations
from feature_engineering import *
from sklearn.preprocessing import LabelEncoder
import gc
from sklearn.metrics import log_loss
import pickle

In [2]:
train = pd.read_pickle(FILE.train_lgb_base.value)
print('train shape is: {}'.format(train.shape))
test = pd.read_pickle(FILE.test_lgb_base.value)
print('test shape is: {}'.format(test.shape))
test['click'] = np.nan

train shape is: (1001650, 71)
test shape is: (40024, 70)


In [3]:
# roun=1
# append_count = pd.read_pickle(FILE.X_fe_agg_count_formater.value.format(roun))
# print(append_count.shape)
# append_timeNext = pd.read_pickle(FILE.X_fe_agg_time_next_formater.value.format(roun))
# print(append_timeNext.shape)
# append_timeCount = pd.read_pickle(FILE.X_fe_agg_time_count_formater.value.format(roun))
# print(append_timeCount.shape)

# X_fe = append_count.merge(append_timeNext,on='instance_id',how='inner')
# X_fe = X_fe.merge(append_timeCount,on='instance_id',how='inner')

X_fe_train_libfm = pd.read_pickle(FILE.X_fe_train_libfm.value)
X_fe_test_libfm = pd.read_pickle(FILE.X_fe_test_libfm.value)
X_fe = pd.concat([X_fe_train_libfm,X_fe_test_libfm])
X_fe = X_fe.reset_index()
print(X_fe.shape)

(1041674, 29)


In [4]:
init_feature = list(test.columns)
search_feature = list(set(X_fe.columns) - set(init_feature))

# combine

In [5]:
train_order = train[['instance_id']].copy()
train_order['ori_order'] = train_order.index
test_order = test[['instance_id']].copy()
test_order['ori_order'] = test_order.index

X_fe_order = X_fe[['instance_id']].copy()
X_fe_order['fe_order'] = X_fe_order.index
X_fe_train = X_fe_order.merge(train_order,on='instance_id',how='inner')
X_fe_train = X_fe_train.sort_values('ori_order',ascending=True)
X_fe_test = X_fe_order.merge(test_order,on='instance_id',how='inner')
X_fe_test = X_fe_test.sort_values('ori_order',ascending=True)
index_order_train = X_fe_train.fe_order.values
index_order_test = X_fe_test.fe_order.values

X_fe_train = X_fe.iloc[index_order_train].copy()
print(X_fe_train.shape)

X_fe_test = X_fe.iloc[index_order_test].copy()
print(X_fe_test.shape)

assert np.sum(X_fe_train.instance_id.values != train.instance_id.values) == 0
assert np.sum(X_fe_test.instance_id.values != test.instance_id.values) == 0

for col in X_fe_train.columns:
    if col == 'instance_id':
        continue
    else:
        train[col] = X_fe_train[col].values
        test[col] = X_fe_test[col].values
print(train.shape)
print(test.shape)

(1001650, 29)
(40024, 29)
(1001650, 99)
(40024, 99)


# LGBM

In [6]:
from sklearn.model_selection import train_test_split
print_to_file = False 
test_run = False 

def get_time(timezone='America/New_York', time_format='%Y-%m-%d %H:%M:%S'):
    from datetime import datetime
    from dateutil import tz

    # METHOD 1: Hardcode zones:
    from_zone = tz.gettz('UTC')
    to_zone = tz.gettz(timezone)

    utc = datetime.utcnow()

    # Tell the datetime object that it's in UTC time zone since 
    # datetime objects are 'naive' by default
    utc = utc.replace(tzinfo=from_zone)

    # Convert time zone
    est = utc.astimezone(to_zone)

    return est.strftime(time_format)

import sys, time
class Logger(object):
    def __init__(self, logtofile=True, logfilename='log'):
        self.terminal = sys.stdout
        self.logfile = "{}_{}.log".format(logfilename, int(time.time()))
        self.logtofile = logtofile

    def write(self, message):
        #         self.terminal.write(message)
        if self.logtofile:
            self.log = open(self.logfile, "a")
            self.log.write('[' + get_time() + '] ' + message)
            self.log.close()

    def flush(self):
        # this flush method is needed for python 3 compatibility.
        # this handles the flush command by doing nothing.
        # you might want to specify some extra behavior here.
        pass


def divert_printout_to_file():
    sys.stdout = Logger(logfilename='logfile')

if print_to_file:
    divert_printout_to_file()  # note: comment this to use pdb

import numpy as np
import pandas as pd
import gc
import time
from contextlib import contextmanager
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score, roc_curve
from sklearn.model_selection import KFold, StratifiedKFold
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

@contextmanager
def timer(title):
    t0 = time.time()
    yield
    print("{} - done in {:.0f}s".format(title, time.time() - t0))

# LightGBM GBDT with KFold or Stratified KFold
# Parameters from Tilii kernel: https://www.kaggle.com/tilii7/olivier-lightgbm-parameters-by-bayesian-opt/code
def kfold_lightgbm(df, train_df, test_df, holdout, num_folds, submission_file_name, fe_img_name, stratified = False, debug= False, colsample=0.67, max_depth=8, num_leaves=31, min_child_samples=20, subsample=0.7, reg_lambda=0.3, lr=0.04, seed=1001, verbose=100, rounds=None, target='click'):
    print(train_df.shape, test_df.shape, holdout.shape)
    print('MEAN: train({}) vs holdout({}): '.format(len(train_df), len(holdout)), train_df[target].mean(), holdout[target].mean())
    # Divide in training/validation and test data
    if df is not None:
        train_df = df[df[target].notnull()]
        test_df = df[df[target].isnull()]
        print("Starting LightGBM. Train shape: {}, test shape: {}".format(train_df.shape, test_df.shape))
        del df
        gc.collect()
    # Cross validation model
    if stratified:
        folds = StratifiedKFold(n_splits= num_folds, shuffle=True, random_state=seed)
    else:
        folds = KFold(n_splits= num_folds, shuffle=True, random_state=seed)
        
    # Create arrays and dataframes to store results
    oof_preds = np.zeros(train_df.shape[0])
    sub_preds = np.zeros(test_df.shape[0])
    holdout_final_preds = np.zeros(holdout.shape[0])
    feature_importance_df = pd.DataFrame()
    feature_importance_gain_df = pd.DataFrame()
    feats = [f for f in train_df.columns if f not in [target,'time','instance_id','index']]
    train_scores = []
    holdout_scores = []
    scores = []
    diff_val_holdout = []
    
    for n_fold, (train_idx, valid_idx) in enumerate(folds.split(train_df[feats], train_df[target])):
#         print('valid index : ',list(valid_idx)[:5])
        train_x, train_y = train_df[feats].iloc[train_idx], train_df[target].iloc[train_idx]
        valid_x, valid_y = train_df[feats].iloc[valid_idx], train_df[target].iloc[valid_idx]
#         print('MEAN: train({}) vs valid({}): '.format(len(train_y), len(valid_y)), np.mean(train_y), np.mean(valid_y))

        clf = LGBMClassifier(
            nthread=12,
            n_estimators=30000,
            learning_rate=lr,
#             max_bin =10000,
#             num_leaves=num_leaves,
#             colsample_bytree=colsample, # 0.67
#             subsample=subsample,
#             subsample_freq=0, ## disable subsampling
#             max_depth=max_depth,
#             reg_alpha=0.65,
#             reg_lambda=reg_lambda,
#             min_split_gain=0.0222415,
#             min_child_weight=39.3259775,
#             min_child_samples=min_child_samples,
            silent=-1,
            verbose=-1, )
        if rounds is not None:
            clf.n_estimators = rounds
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose)
            oof_preds[valid_idx] = clf.predict_proba(valid_x)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats])[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats])[:, 1] 
        else:
            clf.fit(train_x, train_y, eval_set=[(train_x, train_y), (valid_x, valid_y)], 
                eval_metric= 'logloss', verbose=verbose, early_stopping_rounds= 200)
            oof_preds[valid_idx] = clf.predict_proba(valid_x, num_iteration=clf.best_iteration_)[:, 1]
            sub_preds += clf.predict_proba(test_df[feats], num_iteration=clf.best_iteration_)[:, 1] / folds.n_splits
            holdout_preds = clf.predict_proba(holdout[feats], num_iteration=clf.best_iteration_)[:, 1] 
            
        holdout_final_preds += holdout_preds / folds.n_splits
        score = log_loss(valid_y, oof_preds[valid_idx])
        train_score = clf.best_score_['training']['binary_logloss']
        holdout_score = log_loss(holdout[target], holdout_preds)
        diff = abs(score - holdout_score)
        best_rounds = rounds if rounds is not None else clf.best_iteration_
        print('Fold %2d [%5d] AUC : ho: %.6f / te: %.6f / tr: %.6f (diff: %.6f)' % (n_fold + 1, best_rounds, holdout_score, score,  train_score, diff))
        scores.append(score)
        train_scores.append(train_score)
        holdout_scores.append(holdout_score)
        diff_val_holdout.append(diff)
        
        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = feats
        fold_importance_df["importance"] = clf.feature_importances_
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        
        fold_importance_gain_df = pd.DataFrame()
        fold_importance_gain_df["feature"] = feats
        fold_importance_gain_df["importance"] = clf.booster_.feature_importance(importance_type='gain')
        fold_importance_gain_df["fold"] = n_fold + 1
        feature_importance_gain_df = pd.concat([feature_importance_gain_df, fold_importance_gain_df], axis=0)
        
        del clf, train_x, train_y, valid_x, valid_y
        gc.collect()
    holdout_roc = log_loss(holdout[target], holdout_final_preds)
    holdout_mean = np.mean(holdout_scores)
    full_te_mean = np.mean(scores)
    full_tr_mean = np.mean(train_scores)
#     print('Full AUC score %.6f' % roc_auc_score(train_df['TARGET'], oof_preds))
    print('Full HO score %.6f' % holdout_roc)
    print('FULL HO mean {:.6f}, std {:.6f}'.format(holdout_mean, np.std(holdout_scores)))
    print('FULL TE mean {:.6f}, std {:.6f}'.format(full_te_mean, np.std(scores)))
    print('FULL TR mean {:.6f}, std {:.6f}'.format(full_tr_mean, np.std(train_scores)))
    print('FULL DIFF mean {:.6f}, std {:.6f}'.format(np.mean(diff_val_holdout), np.std(diff_val_holdout)))
    # Write submission file and plot feature importance
    if not debug:
        print('saving...')
        test_df['predicted_score'] = sub_preds
        test_df[['instance_id', 'predicted_score']].to_csv(submission_file_name, index= False)
#     if not print_to_file:
#         display_importances(feature_importance_df, fe_img_name)
    feature_importance_df = feature_importance_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    feature_importance_gain_df = feature_importance_gain_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    return feature_importance_df, feature_importance_gain_df,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_final_preds

# Display/plot feature importance
def display_importances(feature_importance_df_, fe_img_name):
    cols = feature_importance_df_[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False)[:40].index
    best_features = feature_importance_df_.loc[feature_importance_df_.feature.isin(cols)]
    plt.figure(figsize=(8, 10))
    sns.barplot(x="importance", y="feature", data=best_features.sort_values(by="importance", ascending=False))
    plt.title('LightGBM Features (avg over folds)')
    plt.tight_layout()
    plt.savefig(fe_img_name+'.png')


def convert_and_save_imp_df(fe_imp_df, dumpfilename):
    fe_imp_df_mean = fe_imp_df[["feature", "importance"]].groupby("feature").mean().sort_values(by="importance", ascending=False).reset_index()
    pickle.dump(fe_imp_df_mean, open(dumpfilename,'wb'))
    
    

def select_holdout(train,test,shift=1,seed=41):
    samples_size = int(len(test) )
    assert shift in range(1,8)
    test_time_min = test.time.min()
    test_time_max = test.time.max()
    train_time_min = train.time.min()
    train_time_max = train.time.max()
    window_lower_bound = test_time_min - shift*60*60*24
    window_upper_bound = test_time_max - shift*60*60*24
    available_train = train[(train.time<=window_upper_bound) & (train.time>=window_lower_bound)]
    holdout = available_train.sample(n=samples_size,random_state=seed).copy()
    train_split_index = list(set(train.index) - set(holdout.index))
    train_split = train.iloc[train_split_index].copy()
    return train_split,holdout

In [7]:
def runlgb(train, test, holdout):
    colsamples = [0.07]#[0.1,0.15,0.2]#[0.03,0.04,0.05,0.06,0.07,0.08]
    seeds = [1001]#[300,4000,50000,600000,7000000,80000000,523445,31275479] # 20
    depth = [5]
    leaves = [16]
    min_child_sam = [20]#, 800]
    subsamples = [1]#0.8, 0.7, 0.6, 0.5, 0.4] # was 1
    reg_lambdas = [0.5]
    # lrs = lrs.tolist()
    lrs2 = [0.1]
    nfolds = 7
    rounds = [None] #[1000]#, 1300, 1600, 1900, 2200, 2500]
    for seed in seeds:
        for colsample in colsamples:
            for d in depth:
                for l in leaves:
                    for mcs in min_child_sam:
                        for subsample in subsamples:
                            for reg_lambda in reg_lambdas:
                                for lr in lrs2:
                                    for r in rounds:
                                        filename = 'fe_936_col{}_lr{}_n{}'.format(len(train.columns), lr, nfolds)
                                        print('#############################################')
                                        print(colsample, seed, d, l, mcs, subsample, reg_lambda, lr, 'nfolds:', nfolds)
                                        print('#############################################')
                                        numfeats = len(train.columns)
                                        with timer("Run LightGBM with kfold"):
                                            return kfold_lightgbm(None, train, test, holdout, nfolds, filename+'.csv', filename, colsample=colsample, verbose=None, max_depth=d, num_leaves=l, min_child_samples=mcs, subsample=subsample, reg_lambda=reg_lambda, lr=lr, seed=seed, stratified=True, rounds=r,debug=True)                                        

In [8]:
# split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)

In [9]:
def add_each_col(train_df,holdout,test,search_feature,target,init_feature,report_path='report_lbfm.csv',by='all'):
    """
    train_df, the train set which does not has holdout
    by = 'cv','ho', other == cv+ ho
    """
    #1 run base line
    
    try:
        report = pd.read_csv(report_path)
        print('found report! resume from last time!')
    except:
        print('calculate importance to generate report...')
        split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)
        fea = [f for f in split_all.feature.values if f in search_feature]
        report = pd.DataFrame({'feature':fea})
        report['order'] = report.index
        report['holdout'] = np.nan
        report['holdout_mean'] = np.nan
        report['search'] = 'n'
        report['add'] = 'n'
        report['cv'] = np.nan
    added = report[report['add']=='y'].copy()
    added_col = []
    if len(added) == 0:
        print('calculate base line...')
        _, _,holdout_base,_,cv_base,_,_,_ = runlgb(train_df[init_feature], 
                                                   test[init_feature], 
                                                   holdout[init_feature])
    else:
        added = added.sort_values('order')
        holdout_base = added.iloc[-1]['holdout']
        cv_base = added.iloc[-1]['cv']
        added_col = list(added['feature'].values)
    print('---- holdout base is {}. cv base is {}'.format(holdout_base,cv_base))
    print('added features are: {}'.format(added_col))
    best_col = init_feature + added_col

    searched = report[report['search']=='y'].copy()
    start_index = len(searched)


    for i in range(start_index,len(report)):
        col = report.iloc[i]['feature']
        print('start... {}/{} fe:{}'.format(i,len(report),col))
        _, _,ho_cur,ho_mean_cur,cv_cur,_,_,_ = runlgb(train_df[best_col+[col]], 
                                                   test[best_col+[col]], 
                                                   holdout[best_col+[col]])
        report.loc[i,'holdout'] = ho_cur
        report.loc[i,'holdout_mean'] = ho_mean_cur
        report.loc[i,'search'] = 'y'
        report.loc[i,'cv'] = cv_cur
        if by == 'cv' and cv_cur < cv_base:
            cv_base = cv_cur
            report.loc[i,'add'] = 'y'
            print('#### {}/{} YES ADD col: {}'.format(i,len(report),col))
            best_col = best_col + [col]
        elif by == 'ho' and ho_cur < holdout_base:
            holdout_base = ho_cur
            report.loc[i,'add'] = 'y'
            print('#### {}/{} YES ADD col: {}'.format(i,len(report),col))
            best_col = best_col + [col]
        elif cv_cur < cv_base and ho_cur < holdout_base:
            cv_base = cv_cur
            holdout_base = ho_cur
            report.loc[i,'add'] = 'y'
            print('#### {}/{} YES ADD col: {}'.format(i,len(report),col))
            best_col = best_col + [col]
        else:
            report.loc[i,'add'] = 'n'
            print('#### {}/{} NO ADD col: {}'.format(i,len(report),col))
        report.to_csv(report_path,index=False)
        print('')

In [10]:
report_path = 'report_lbfm.csv'
target='click'
train_index = pickle.load(open(FILE.train_index.value,'rb'))
holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
train_df = train.loc[train_index].copy()
holdout = train.loc[holdout_index].copy()
print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
print('train mean: {}, holdout mean: {}'.format(train_df[target].mean(),holdout[target].mean()))

add_each_col(train_df,holdout,test,search_feature,target,init_feature,report_path='report_libfm_holdout.csv',by='ho')

train shape: (961626, 99), holdout shape: (40024, 99)
train mean: 0.19797613625255556, holdout mean: 0.21007395562662404
calculate importance to generate report...
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 99) (40024, 99) (40024, 99)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  235] AUC : ho: 0.426204 / te: 0.416188 / tr: 0.407362 (diff: 0.010016)
Fold  2 [  367] AUC : ho: 0.425882 / te: 0.415554 / tr: 0.402780 (diff: 0.010329)
Fold  3 [  433] AUC : ho: 0.425970 / te: 0.415725 / tr: 0.400690 (diff: 0.010244)
Fold  4 [  400] AUC : ho: 0.425705 / te: 0.415906 / tr: 0.401545 (diff: 0.009799)
Fold  5 [  284] AUC : ho: 0.425984 / te: 0.417214 / tr: 0.405328 (diff: 0.008770)
Fold  6 [  277] AUC : ho: 0.425762 / te: 0.416537 / tr: 0.405667 (diff: 0.009225)
Fold  7 [  116] AUC : ho: 0.426008 / te: 0.416292 / tr: 0.411733 (diff: 0.009717)
Full HO score 0

Fold  1 [  356] AUC : ho: 0.425882 / te: 0.416044 / tr: 0.405394 (diff: 0.009838)
Fold  2 [  431] AUC : ho: 0.425931 / te: 0.415291 / tr: 0.403437 (diff: 0.010640)
Fold  3 [  422] AUC : ho: 0.425901 / te: 0.415437 / tr: 0.403552 (diff: 0.010465)
Fold  4 [  313] AUC : ho: 0.425669 / te: 0.415956 / tr: 0.406567 (diff: 0.009713)
Fold  5 [  521] AUC : ho: 0.426327 / te: 0.417098 / tr: 0.401016 (diff: 0.009229)
Fold  6 [  417] AUC : ho: 0.425764 / te: 0.416133 / tr: 0.403584 (diff: 0.009630)
Fold  7 [  257] AUC : ho: 0.425817 / te: 0.416277 / tr: 0.408104 (diff: 0.009540)
Full HO score 0.425264
FULL HO mean 0.425899, std 0.000194
FULL TE mean 0.416034, std 0.000550
FULL TR mean 0.404522, std 0.002173
FULL DIFF mean 0.009865, std 0.000470
Run LightGBM with kfold - done in 190s
#### 5/27 YES ADD col: uid1+uraw1+itemc3_matrix_factorization

start... 6/27 fe:uid1+uraw2+itemc2_matrix_factorization
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
###############

Fold  3 [  357] AUC : ho: 0.425713 / te: 0.415495 / tr: 0.404404 (diff: 0.010218)
Fold  4 [  461] AUC : ho: 0.425818 / te: 0.415960 / tr: 0.401577 (diff: 0.009858)
Fold  5 [  636] AUC : ho: 0.426066 / te: 0.416938 / tr: 0.396833 (diff: 0.009128)
Fold  6 [  407] AUC : ho: 0.425705 / te: 0.416372 / tr: 0.402966 (diff: 0.009333)
Fold  7 [  215] AUC : ho: 0.425967 / te: 0.416176 / tr: 0.408864 (diff: 0.009791)
Full HO score 0.425176
FULL HO mean 0.425837, std 0.000123
FULL TE mean 0.416060, std 0.000480
FULL TR mean 0.403942, std 0.003973
FULL DIFF mean 0.009778, std 0.000405
Run LightGBM with kfold - done in 183s
#### 12/27 NO ADD col: uid2+uraw3+itemc1_matrix_factorization

start... 13/27 fe:uid3+uraw1+itemc3_matrix_factorization
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 78) (40024, 78) (40024, 78)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  325]

Fold  5 [  186] AUC : ho: 0.425385 / te: 0.417220 / tr: 0.409476 (diff: 0.008165)
Fold  6 [  467] AUC : ho: 0.425845 / te: 0.416210 / tr: 0.400859 (diff: 0.009634)
Fold  7 [  534] AUC : ho: 0.426291 / te: 0.416210 / tr: 0.399224 (diff: 0.010081)
Full HO score 0.425104
FULL HO mean 0.425713, std 0.000279
FULL TE mean 0.416104, std 0.000518
FULL TR mean 0.404886, std 0.003813
FULL DIFF mean 0.009609, std 0.000608
Run LightGBM with kfold - done in 189s
#### 19/27 NO ADD col: uid3+uraw1+itemc1_matrix_factorization

start... 20/27 fe:uid1+uraw3+itemc2_matrix_factorization
#############################################
0.07 1001 5 16 20 1 0.5 0.1 nfolds: 7
#############################################
(961626, 81) (40024, 81) (40024, 81)
MEAN: train(961626) vs holdout(40024):  0.19797613625255556 0.21007395562662404
Fold  1 [  378] AUC : ho: 0.425490 / te: 0.415838 / tr: 0.403559 (diff: 0.009652)
Fold  2 [  395] AUC : ho: 0.425326 / te: 0.415477 / tr: 0.403001 (diff: 0.009849)
Fold  3 [  408]

Fold  7 [  318] AUC : ho: 0.425844 / te: 0.416105 / tr: 0.405310 (diff: 0.009739)
Full HO score 0.425125
FULL HO mean 0.425735, std 0.000201
FULL TE mean 0.416080, std 0.000486
FULL TR mean 0.404368, std 0.002612
FULL DIFF mean 0.009655, std 0.000654
Run LightGBM with kfold - done in 187s
#### 26/27 NO ADD col: uid2+uraw1+itemc1_matrix_factorization



In [15]:
report = pd.read_csv('report_libfm_holdout.csv')
added = list(report[report['add']=='y'].feature.values)

In [18]:
saved_col = init_feature + added
train_save = train[saved_col].copy()
print(train_save.shape)
test_save = test[saved_col].copy()
print(train_save.shape)
train_save.to_pickle('../../data/lgb_feature/train_lgb_libFM_ho.pickle')
test_save.to_pickle('../../data/lgb_feature/test_lgb_libFM_ho.pickle')

(1001650, 81)
(1001650, 81)


In [11]:
# #1 run base line
# report_path = 'report_lbfm.csv'
# target='click'
# train_index = pickle.load(open(FILE.train_index.value,'rb'))
# holdout_index = pickle.load(open(FILE.holdout_index.value,'rb'))
# train_df = train.loc[train_index].copy()
# holdout = train.loc[holdout_index].copy()
# print('train shape: {}, holdout shape: {}'.format(train_df.shape,holdout.shape))
# print('train mean: {}, holdout mean: {}'.format(train_df[target].mean(),holdout[target].mean()))
# try:
#     report = pd.read_csv(report_path)
#     print('found report! resume from last time!')
# except:
#     print('calculate importance to generate report...')
#     split_all, gain_all,holdout_roc,holdout_mean,full_te_mean,full_tr_mean,oof_preds,holdout_preds = runlgb(train_df, test, holdout)
#     fea = [f for f in split_all.feature.values if f in search_feature]
#     report = pd.DataFrame({'feature':fea})
#     report['order'] = report.index
#     report['holdout'] = np.nan
#     report['holdout_mean'] = np.nan
#     report['search'] = 'n'
#     report['add'] = 'n'
#     report['cv'] = np.nan
# added = report[report['add']=='y'].copy()
# added_col = []
# if len(added) == 0:
#     print('calculate base line...')
#     _, _,holdout_base,_,cv_base,_,_,_ = runlgb(train_df[init_feature], 
#                                                test[init_feature], 
#                                                holdout[init_feature])
# else:
#     added = added.sort_values('order')
#     holdout_base = added.iloc[-1]['holdout']
#     cv_base = added.iloc[-1]['cv']
#     added_col = list(added['feature'].values)
# print('---- holdout base is {}. cv base is {}'.format(holdout_base,cv_base))
# print('added features are: {}'.format(added_col))
# best_col = init_feature + added_col

# searched = report[report['search']=='y'].copy()
# start_index = len(searched)


# for i in range(start_index,len(report)):
#     print('start... {}/{} fe:{}'.format(i,len(report),col))
#     col = report.iloc[i]['feature']
#     _, _,ho_cur,ho_mean_cur,cv_cur,_,_,_ = runlgb(train_df[best_col+[col]], 
#                                                test[best_col+[col]], 
#                                                holdout[best_col+[col]])
#     report.loc[i,'holdout'] = ho_cur
#     report.loc[i,'holdout_mean'] = ho_mean_cur
#     report.loc[i,'search'] = 'y'
#     report.loc[i,'cv'] = cv_cur
#     if cv_cur < cv_base and ho_cur < holdout_base:
#         cv_base = cv_cur
#         holdout_base = ho_cur
#         report.loc[i,'add'] = 'y'
#         print('#### {}/{} YES ADD col: {}'.format(i,len(report),col))
#         best_col = best_col + [col]
#     else:
#         report.loc[i,'add'] = 'n'
#         print('#### {}/{} NO ADD col: {}'.format(i,len(report),col))
#     report.to_csv(report_path,index=False)
#     print('')

    

    

In [12]:
## 24	osv_count_in_next_n_time_unit_6hour	24	0.426921	0.42762	y	y	0.416452